In [18]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
df = pd.read_csv('data.csv')
descriptions = df['Description'].tolist()
categories = df['Genre'].tolist()
names = df['title'].tolist()

In [20]:
df2=df.dropna().reset_index(drop=True)

In [ ]:
vectors = [

  model(**tokenizer(description, return_tensors='pt', truncation=True, padding = True))[0].detach().squeeze()
  for description in descriptions
]
[v.size() for v in vectors]

#randam vektoriu vidurkius
averaged_vectors = [
    torch.mean(vector, dim=0) 
    for vector in vectors
    ]
#[v.size() for v in averaged_vectors]

#print(averaged_vectors)

probs = [t.detach().numpy() for t in averaged_vectors]


with open('Vectors_names.pkl', 'wb') as f:
    pickle.dump(probs, f)

In [6]:

vectors_of_category = [

  model(**tokenizer(category, return_tensors='pt', truncation=True, padding = True))[0].detach().squeeze()
  for category in categories
]
#[v.size() for v in vectors]

#randam vektoriu vidurkius
averaged_vectors_of_category = [
    torch.mean(vector_of_category, dim=0) 
    for vector_of_category in vectors_of_category
    ]
#[v.size() for v in averaged_vectors]

#print(averaged_vectors)

probs_of_category = [t.detach().numpy() for t in averaged_vectors_of_category]

with open('Vectors_category.pkl', 'wb') as f:
    pickle.dump(probs_of_category, f)

In [36]:

name = 'NBA'
def games_recommender(name):
    with open('Vectors_names.pkl', 'rb') as f:
     probs = pickle.load(f)



    vector_of_name = model(**tokenizer(name, return_tensors='pt', truncation=True, padding = True))[0].detach().squeeze()

    similarity = cosine_similarity(vector_of_name, probs)

    games = []
    for arr in similarity:
      for i, each_val in enumerate(arr):
         games.append([names[i],each_val])

    final_games = sorted(games, key=lambda x: x[1], reverse=True)

    for arr in final_games[0:1]:
    # print(f'\nScore : \n\n  {arr[1]}')
      print(f'\nGame : \n\n {arr[0]}')

    arr = np.delete(arr,-1)

    arr = ''.join(arr)

    with open('Vectors1.pkl', 'rb') as f:
     probs1 = pickle.load(f)


    similarity2 = cosine_similarity(probs1)

    indexes = pd.Series(df2['title'])

    similar_games = []
    idx = indexes[indexes == arr].index[0]  
    similarities = pd.Series(similarity2[idx]).sort_values(ascending = False) 
    indexes_count = list(similarities.iloc[1:5].index)

    for i in indexes_count:  
        similar_games.append(list(df2['title'])[i])

    return similar_games


print(games_recommender(name))

    


Game : 

 NBA 2K22: NBA 75th Anniversary Edition XBOX LIVE Key UNITED STATES
['NBA 2K22 (PC) Steam Key NORTH AMERICA', 'NBA 2K22 (Xbox Series X|S) Xbox Live Key UNITED STATES', 'NBA 2K23 Michael Jordan Edition (Xbox One/Xbox Series S|X) Key UNITED STATES', 'NBA 2K23 for Xbox One Key GLOBAL']


In [41]:
category = 'sport'


def get_games_by_category(category):
    with open('Vectors_category.pkl', 'rb') as f:
     probs = pickle.load(f)

    vector_of_category = model(**tokenizer(category, return_tensors='pt', truncation=True, padding = True))[0].detach().squeeze()

    similarity_of_category = cosine_similarity(vector_of_category, probs)

    categor = []
    for arr_of_category in similarity_of_category:
      for i, each_val in enumerate(arr_of_category):
          categor.append([categories[i],each_val])

    final_categor = sorted(categor, key=lambda x: x[1], reverse=True)

    for arr_of_category in final_categor[0:1]:
      print(f'\ncategory : \n\n {arr_of_category[0]}')

    arr_of_category = np.delete(arr_of_category,-1)

    arr_of_category = ''.join(arr_of_category)

    similarity_of_category2 = cosine_similarity(probs)

    indexes = pd.Series(df2['Genre'])

    similar_games = []
    Prices = []
    idx = indexes[indexes == arr_of_category].index[0]  
    similarities = pd.Series(similarity_of_category2[idx]).sort_values(ascending = False) 
    indexes_count = list(similarities.iloc[1:11].index)  

    for i in indexes_count:  
        similar_games.append(list(df2['title'])[i])
        #similar_games.append(list(df2['Genre'])[i])
        Prices.append(list(df2['Price'])[i])

    Price = list(map(lambda x: x.strip('€'), Prices))

   

    #Price = filter(None, Price)

    #print(Price)

    foo = [float(i) for i in Price]

    avg = statistics.mean(foo)

    avg=str(avg)

    similar_games = str(similar_games)

    

    print('average price of games in this category is:\n' + avg + "€")

    print('games in this category\n'+ category + '\nare these:\n')

    return similar_games

print(get_games_by_category(category))



    



category : 

 Sport
['79.95', '57.99', '54.46', '15.66', '48.78', '11.99', '9.99', '20.10', '9.08', '20.07']
average price of games in this category is:
32.807€
games in this category
sport
are these:

['NHL® 23 X-Factor Edition Xbox One & Xbox Series X|S Key UNITED STATES', 'NHL® 23 Standard Edition Xbox Series X|S Key UNITED STATES', 'Mario Strikers: Battle League (Nintendo Switch) eShop Key UNITED STATES', 'The Evil Within Digital Bundle XBOX LIVE Key UNITED STATES', 'Gotham Knights (PC) Steam Key GLOBAL', 'Resident Evil 3 (Xbox One) Xbox Live Key UNITED STATES', 'GUILTY GEAR (PC) Steam Key GLOBAL', 'Resident Evil Revelations 1 & 2 Bundle XBOX LIVE Key UNITED STATES', 'Warhammer 40,000: Space Marine - Anniversary Edition (PC) Steam Key GLOBAL', 'GRID Legends XBOX LIVE Key UNITED STATES']
